In [1]:
from IPython.display import display, Image, clear_output
import PIL.Image
import io

In [2]:
import cv2
import time
import math as m
import mediapipe as mp

In [3]:
# Initilize medipipe selfie segmentation class.
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic

In [4]:
def findDistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2+(y2-y1)**2)
    return dist

In [5]:
# Calculate angle.
def findAngle(x1, y1, x2, y2):
    theta = m.acos((y2 -y1)*(-y1) / (m.sqrt((x2 - x1)**2 + (y2 - y1)**2) * y1))
    degree = int(180/m.pi)*theta
    return degree

In [6]:
def sendWarning(image):
    print("Alert! Poor posture detected!")
    cv2.putText(image, "WARNING: Poor Posture!", (50, 50), font, 1.5, red, 3)

In [7]:
# Initilize frame counters.
good_frames = 0
bad_frames = 0

# Font type.
font = cv2.FONT_HERSHEY_SIMPLEX

# Colors.
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)

In [8]:
# Initialize mediapipe pose class.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize video capture object.
# For webcam input replace file name with 0.
cap = cv2.VideoCapture(0)

# Meta.
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Initialize video writer.
video_output = cv2.VideoWriter('output.mp4', fourcc, fps, frame_size)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [9]:
print(f"FPS: {fps}")

FPS: 30


In [10]:
print("Frame Captured")

Frame Captured


In [11]:
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Null.Frames")
        break
    
    # Get fps.
    fps = cap.get(cv2.CAP_PROP_FPS)
    h, w = image.shape[:2]

    # Convert the BGR image to RGB.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image.
    keypoints = pose.process(image)

    # Convert the image back to BGR.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Get the landmarks.
    lm = keypoints.pose_landmarks
    lmPose = mp_pose.PoseLandmark

    if lm:
        # Acquire coordinates for shoulder, ear, and hip landmarks.
        l_shldr_x = int(lm.landmark[lmPose.LEFT_SHOULDER].x * w)
        l_shldr_y = int(lm.landmark[lmPose.LEFT_SHOULDER].y * h)
        r_shldr_x = int(lm.landmark[lmPose.RIGHT_SHOULDER].x * w)
        r_shldr_y = int(lm.landmark[lmPose.RIGHT_SHOULDER].y * h)
        l_ear_x = int(lm.landmark[lmPose.LEFT_EAR].x * w)
        l_ear_y = int(lm.landmark[lmPose.LEFT_EAR].y * h)
        l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
        l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)

        # Calculate the distances and angles.
        offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        neck_inclination = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
        torso_inclination = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

        # Define the angle text string.
        angle_text_string = 'Neck : ' + str(int(neck_inclination)) + '  Torso : ' + str(int(torso_inclination))

        # Draw the key points and lines.
        # --- [Draw landmarks and lines code here, unchanged] ---

        # Update the posture status based on the calculated angles.
        if neck_inclination < 40 and torso_inclination < 10:
            bad_frames = 0
            good_frames += 1
            cv2.putText(image, angle_text_string, (10, 30), font, 0.9, light_green, 2)
        else:
            good_frames = 0
            bad_frames += 1
            cv2.putText(image, angle_text_string, (10, 30), font, 0.9, red, 2)

        # Calculate the time spent in good or bad posture.
        good_time = (1 / fps) * good_frames
        bad_time = (1 / fps) * bad_frames

        # Display the posture time.
        if good_time > 0:
            time_string_good = 'Good Posture Time : ' + str(round(good_time, 1)) + 's'
            cv2.putText(image, time_string_good, (10, h - 20), font, 0.9, green, 2)
        else:
            time_string_bad = 'Bad Posture Time : ' + str(round(bad_time, 1)) + 's'
            cv2.putText(image, time_string_bad, (10, h - 20), font, 0.9, red, 2)

        # Send a warning if bad posture persists for more than 30 seconds.
        if bad_time > 30:
            sendWarning()
            print(f"Bad time: {bad_time}s")

    # Write the processed frame to the output video.
    video_output.write(image)

print('Finished.')
cap.release()
video_output.release()

KeyboardInterrupt: 

In [ ]:
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Null.Frames")
        break

    fps = cap.get(cv2.CAP_PROP_FPS)
    h, w = image.shape[:2]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    keypoints = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    lm = keypoints.pose_landmarks
    lmPose = mp_pose.PoseLandmark

    if lm:
        # Process the posture logic...
        if bad_time > 30:
            sendWarning(image)  # Overlay warning on the frame

    # Convert the frame to display in Jupyter
    _, buffer = cv2.imencode('.png', image)
    frame = PIL.Image.open(io.BytesIO(buffer))
    
    # Display in Jupyter
    clear_output(wait=True)
    display(frame)

    # Write the processed frame to the output video
    video_output.write(image)

cap.release()
video_output.release()